## 250430 테스트
* HyperClovaX 테스트(임베딩 및 Chat Model 확인)
* Milvus DB 구성 요소 생성

In [1]:
import os
import re
import json
import jsonlines
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_milvus.vectorstores import Milvus
from uuid import uuid4
from langchain_community.chat_models import ChatClovaX
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import pandas as pd
import pytz

from datasets import Dataset
from datetime import timedelta
from operator import itemgetter
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import (
  AttributeInfo,
  StructuredQueryOutputParser,
  get_query_constructor_prompt
)
from langchain_teddynote.evaluator import GroundednessChecker
from langchain.retrievers.self_query.milvus import MilvusTranslator
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
import warnings
from langchain_core.runnables import chain

warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
embeddings = ClovaXEmbeddings(
    model='bge-m3'
)

In [6]:
def open_jsonl(filepath):
    test_docs = []
    with open(filepath, 'r', encoding='utf-8') as file:
      for line in file:
        # 각 줄을 JSON으로 파싱
        data = json.loads(line)
        
        # Document 객체 생성 및 리스트에 추가
        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        test_docs.append(doc)
    return test_docs

In [ ]:
URI = 'http://127.0.0.1:19530'

text_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_db'
)

In [7]:
docs = open_jsonl('./chunked_jsonl/text_semantic_per_80.jsonl')

In [8]:
docs

[Document(metadata={'type': 'text', 'page': 8, 'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'summary': '- 국공채 시장은 약세로 시작, 외국인 매수세로 일부 회복.\n- 호주 5월 CPI가 전년 대비 4.0% 상승, 예상치 초과로 호주 국채 약세.\n- 국고채 20년 금리는 3.220%에 7천억 원 낙찰, 2조 1720억 원 응찰.\n- 28일 국고채 3년 최종호가는 3.182%, 10년물은 3.266%로 각각 3.8bp, 3.4bp 하락.\n- 국고 3년물은 지난주 대비 1.7bp 상승, 5년물은 0.5bp 상승, 10년물은 1.8bp 상승.\n- 국고 3/5년 스프레드는 1.8bp로 1.2bp 축소, 5/10년 스프레드는 6.5bp, 3/10년 스프레드는 8.3bp로 0.1bp 확대.', 'issue_date': '20240628'}, page_content='주 첫 영업일 국내 채권시장은 지난주 급격하게 하락한 금리를 되돌리며 약세로 출발했다. 장중 외 국인 매수세 지속으로 약세가 일부 되돌려졌으나, 전반적인 아시아 채권시장의 약세 분위기로 국내 채 권시장 또한 약세 우위를 보였다. 익영업일 미국 채권시장 강세의 영향으로 국고채 금리 또한 하락 출 발했다.'),
 Document(metadata={'type': 'text', 'page': 8, 'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'summary': '- 국공채 시장은 약세로 시작, 외국인 매수세로 일부 회복.\n- 호주 5월 CPI가 전년 대비 4.0% 상승, 예상치 초과로 호주 국채 약세.\n- 국고채 20년 금리는 3.220%에 7천억 원 낙찰, 2조 1720억 원 응찰.\n- 28일 국고채 3년 최종호가는 3.182%, 10년물은 3.266%로 각각 3.8bp, 3.4bp 하락.\n- 국고 3년물은 지난주 대비 1.7bp 상

In [9]:
uuids = [str(uuid4()) for _ in range(len(docs))]
text_db.add_documents(
    documents=docs,
    ids=uuids
)

HTTPStatusError: Error response 429 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/api-tools/embedding/v2: {"status":{"code":"42901","message":"Too many requests - rate exceeded"},"result":null}